In [4]:

import jpype.imports;
from jpype.types import JString
import ipywidgets as widgets
from IPython.display import display, clear_output

# Start the JVM
jpype.startJVM()

# Java code for the calculator
java_code = """
public class Calculator {
    public static String calculate(String expression) {
        try {
            // Simple evaluation of basic arithmetic expressions
            return String.valueOf(new Object() {
                int pos = -1, ch;

                void nextChar() {
                    ch = (++pos < expression.length()) ? expression.charAt(pos) : -1;
                }

                boolean eat(int charToEat) {
                    while (ch == ' ') nextChar();
                    if (ch == charToEat) {
                        nextChar();
                        return true;
                    }
                    return false;
                }

                double parse() {
                    nextChar();
                    double x = parseExpression();
                    if (pos < expression.length()) throw new RuntimeException("Unexpected: " + (char) ch);
                    return x;
                }

                double parseExpression() {
                    double x = parseTerm();
                    for (;;) {
                        if (eat('+')) x += parseTerm();
                        else if (eat('-')) x -= parseTerm();
                        else return x;
                    }
                }

                double parseTerm() {
                    double x = parseFactor();
                    for (;;) {
                        if (eat('*')) x *= parseFactor();
                        else if (eat('/')) x /= parseFactor();
                        else return x;
                    }
                }

                double parseFactor() {
                    if (eat('+')) return parseFactor();
                    if (eat('-')) return -parseFactor();

                    double x;
                    int startPos = this.pos;
                    if (eat('(')) {
                        x = parseExpression();
                        eat(')');
                    } else if ((ch >= '0' && ch <= '9') || ch == '.') {
                        while ((ch >= '0' && ch <= '9') || ch == '.') nextChar();
                        x = Double.parseDouble(expression.substring(startPos, this.pos));
                    } else {
                        throw new RuntimeException("Unexpected: " + (char) ch);
                    }
                    return x;
                }
            }.parse());
        } catch (Exception e) {
            return "Error";
        }
    }
}
"""

# Create a Java class from the string
jpype.java.lang.System.setProperty("java.class.path", ".")
jpype.java.lang.System.out.println("Loading Java Calculator Class")
jpype.JClass("java.lang.Compiler").compile(JString(java_code), "Calculator")

# Create UI components
expression_input = widgets.Text(description='Expression:')
output_area = widgets.Output()

def on_calculate(change):
    with output_area:
        clear_output()
        expression = expression_input.value
        result = jpype.JClass("Calculator").calculate(JString(expression))
        print(f"Result: {result}")

expression_input.observe(on_calculate, names='value')
display(expression_input, output_area)


CompilationException: 